In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path, PosixPath
from tqdm import tqdm
import sys
import glob
import datetime
from typing import List
import json

In [2]:
CREATE_CSV_FILES = True
CAMELS_COMBINED = True
COMPARE_SOURCES = True
root_path = "/content/gdrive/MyDrive/Colab Datasets/Hydrology/cl"

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Create csv Files



## Read Timeseries Files

In [3]:
def to_datetime(s):
  date = s.split('-')
  year = int(date[0])
  month = int(date[1])
  day = int(date[2])
  d = datetime.date(year, month, day)
  return d


def read_timeseries_data(data_dir: str):
  files = list(Path(data_dir).glob('*.csv'))
  input = []

  for csv in files:
      file_name = os.path.basename(csv)
      names = file_name.split('.')

      df_input = pd.read_csv(csv, dtype=None)
      df_input['basin_id'] = int(names[0])
      df_input['date'].apply(to_datetime)
      input.append(df_input)

  timeseries_data = pd.concat(input, axis=0, copy=False)

  return timeseries_data

In [4]:
if CREATE_CSV_FILES:
  timeseries_dir = root_path + "/raw_timeseries/preprocessed"
  timeseries_data = read_timeseries_data(timeseries_dir)
  forcing_data = timeseries_data[['date', 'basin_id', 'precip_cr2met', 'tmean_cr2met']]
  discharge_data = timeseries_data[['date', 'basin_id', 'streamflow_mm']]

  forcing_data.to_csv(root_path + "/forcing_data_cl.csv", index=False)
  discharge_data.to_csv(root_path + "/discharge_data_cl.csv", index=False)

## Read Attributes Files

In [7]:
def read_attributes(data_dir: str):
  attributes = pd.read_csv(data_dir + "/CAMELS_cl_attributes.csv")
  attributes = attributes.astype({"gauge_id": int}, errors='raise')

  return attributes

In [8]:
if CREATE_CSV_FILES:
  attributes_dir = root_path
  attr = read_attributes(attributes_dir)
  attr.to_csv(root_path + "/attributes_cl.csv", index=False)
  attr

# Read csv files

In [9]:
BASIN_COL = 'basin_id'
DATE_COL = 'date'

In [10]:
# read csvs
attr_p = pd.read_csv(root_path + "/attributes_cl.csv")
discharge_p = pd.read_csv(root_path + "/discharge_data_cl.csv")
forcing_p = pd.read_csv(root_path + "/forcing_data_cl.csv")

In [11]:
discharge_p

,date,basin_id,streamflow_mm
0,1976-07-21,1001001,0.870746
1,1976-07-22,1001001,0.863794
2,1976-07-23,1001001,0.841200
3,1976-07-24,1001001,0.802963
4,1976-07-25,1001001,0.782107
...,...,...,...
9893832,2018-03-05,12930001,1.021117
9893833,2018-03-06,12930001,0.949974
9893834,2018-03-07,12930001,0.929049
9893835,2018-03-08,12930001,0.929049


In [12]:
forcing_p

,date,basin_id,precip_cr2met,tmean_cr2met
0,1976-07-21,1001001,NaN,NaN
1,1976-07-22,1001001,NaN,NaN
2,1976-07-23,1001001,NaN,NaN
3,1976-07-24,1001001,NaN,NaN
4,1976-07-25,1001001,NaN,NaN
...,...,...,...,...
9893832,2018-03-05,12930001,NaN,NaN
9893833,2018-03-06,12930001,NaN,NaN
9893834,2018-03-07,12930001,NaN,NaN
9893835,2018-03-08,12930001,NaN,NaN


In [13]:
# read csvs
attr_p = pd.read_csv(root_path + "/attributes_cl.csv")
discharge_p = pd.read_csv(root_path + "/discharge_data_cl.csv")
forcing_p = pd.read_csv(root_path + "/forcing_data_cl.csv")

In [14]:
locs = np.union1d(np.union1d(forcing_p[BASIN_COL], discharge_p[BASIN_COL]), attr_p['gauge_id'])
Nloc = len(locs)

print('Initial Nloc', Nloc)

Initial Nloc 516


## Sanity Check

In [15]:
# Compare precipitation measurement sources
if COMPARE_SOURCES:
  timeseries_dir = root_path + "/raw_timeseries/preprocessed"
  timeseries_data = read_timeseries_data(timeseries_dir)

  # cr2met & chirps comparison
  precip_crch = timeseries_data[['precip_cr2met', 'precip_chirps']].dropna(axis = 0).astype(float)
  n1 = precip_crch.shape[0]
  precip_crch['cr^2'] = precip_crch['precip_cr2met'] ** 2
  precip_crch['ch^2'] = precip_crch['precip_chirps'] ** 2
  precip_crch['crch'] = precip_crch['precip_cr2met'] * precip_crch['precip_chirps']

  numerator1 = n1 * (precip_crch['crch'].sum()) - (precip_crch['precip_cr2met'].sum() * precip_crch['precip_chirps'].sum())
  denominator1 = (n1 * precip_crch['cr^2'].sum() - (precip_crch['precip_cr2met'].sum() ** 2)) * (n1 * precip_crch['ch^2'].sum() - (precip_crch['precip_chirps'].sum() ** 2))
  r1 = numerator1/(denominator1**0.5)

  print('Correlation coefficient for cr2met and chirps is:', r1)


  # cr2met & tmpa comparison
  precip_crtm = timeseries_data[['precip_cr2met', 'precip_tmpa']].dropna(axis = 0).astype(float)
  n2 = precip_crtm.shape[0]
  precip_crtm['cr^2'] = precip_crtm['precip_cr2met'] ** 2
  precip_crtm['tm^2'] = precip_crtm['precip_tmpa'] ** 2
  precip_crtm['crtm'] = precip_crtm['precip_cr2met'] * precip_crtm['precip_tmpa']

  numerator2 = n2 * (precip_crtm['crtm'].sum()) - (precip_crtm['precip_cr2met'].sum() * precip_crtm['precip_tmpa'].sum())
  denominator2 = (n2 * precip_crtm['cr^2'].sum() - (precip_crtm['precip_cr2met'].sum() ** 2)) * (n2 * precip_crtm['tm^2'].sum() - (precip_crtm['precip_tmpa'].sum() ** 2))
  r2 = numerator2/(denominator2**0.5)

  print('Correlation coefficient for cr2met and tmpa is:', r2)

  # cr2met & mswep comparison
  precip_crms = timeseries_data[['precip_cr2met', 'precip_mswep']].dropna(axis = 0).astype(float)
  n3 = precip_crms.shape[0]
  precip_crms['cr^2'] = precip_crms['precip_cr2met'] ** 2
  precip_crms['ms^2'] = precip_crms['precip_mswep'] ** 2
  precip_crms['crms'] = precip_crms['precip_cr2met'] * precip_crms['precip_mswep']

  numerator3 = n3 * (precip_crms['crms'].sum()) - (precip_crms['precip_cr2met'].sum() * precip_crms['precip_mswep'].sum())
  denominator3 = (n3 * precip_crms['cr^2'].sum() - (precip_crms['precip_cr2met'].sum() ** 2)) * (n3 * precip_crms['ms^2'].sum() - (precip_crms['precip_mswep'].sum() ** 2))
  r3 = numerator3/(denominator3**0.5)

  print('Correlation coefficient for cr2met and mswep is:', r3)

Correlation coefficient for cr2met and chirps is: 0.5735014972080532
Correlation coefficient for cr2met and tmpa is: 0.5930272204337096
Correlation coefficient for cr2met and mswep is: 0.7024249966306338


In [16]:
def remove_basins_globally(basins: List[int]):
  global discharge_p
  global forcing_p
  global attr_p
  global locs
  global Nloc

  print("removing", basins)

  discharge_p.drop(discharge_p[discharge_p[BASIN_COL].isin(basins)].index, inplace=True, errors='ignore')
  forcing_p.drop(forcing_p[forcing_p[BASIN_COL].isin(basins)].index, inplace=True, errors='ignore')
  attr_p.drop(attr_p[attr_p['gauge_id'].isin(basins)].index, inplace=True, errors='ignore')

  locs = np.setdiff1d(locs, basins)
  Nloc = len(locs)
  print('new Nloc', Nloc)

In [17]:
intersect = np.intersect1d(np.intersect1d(forcing_p[BASIN_COL], discharge_p[BASIN_COL]), attr_p['gauge_id'])

diff = np.setdiff1d(locs, intersect)
print("diff", diff)

if len(diff) > 0:
  remove_basins_globally(diff)

diff []


In [18]:
def find_time_range(df: pd.DataFrame):
  groups = df[[BASIN_COL, DATE_COL]].groupby([BASIN_COL])
  counts = groups.count()

  abnormal_cols = counts[counts[DATE_COL] < 1000].index.values

  if len(abnormal_cols) > 0:
    print('abnormal cols', abnormal_cols)
    remove_basins_globally(abnormal_cols)
    groups = df[[BASIN_COL, DATE_COL]].groupby([BASIN_COL])

  mins = groups.min()
  maxs = groups.max()

  min_d  = np.max(mins.values)
  max_d  = np.min(maxs.values)
  print("range", min_d, max_d)

  return min_d, max_d

In [19]:
min1, max1 = find_time_range(discharge_p)
min2, max2 = find_time_range(forcing_p)

range 1979-01-01 2016-12-31
range 1979-01-01 2016-12-31


In [20]:
InitialDate = max(min1, min2)
InitialDate = "1989-10-02"
print("InitialDate ", InitialDate)

EndDate = min(max1, max2)
EndDate = "2008-12-31"
print("EndDate ", EndDate)

InitialDate  1989-10-02
EndDate  2008-12-31


In [21]:
# filter data sets based on the InitialDate and EndDate
discharge_p.drop(discharge_p[(discharge_p[DATE_COL] < InitialDate) | (discharge_p[DATE_COL] > EndDate)].index,
                 inplace=True, errors='ignore')
forcing_p.drop(forcing_p[(forcing_p[DATE_COL] < InitialDate) | (forcing_p[DATE_COL] > EndDate)].index,
                 inplace=True, errors='ignore')

## Dynamic Data

In [22]:
input_p = forcing_p.merge(discharge_p, on=[BASIN_COL, DATE_COL], copy=True, validate="1:1")
input_p

,date,basin_id,precip_cr2met,tmean_cr2met,streamflow_mm
0,1989-10-02,1001001,0.000000,3.767656,0.867270
1,1989-10-03,1001001,0.000000,4.097199,0.865532
2,1989-10-04,1001001,0.000000,3.866711,0.869008
3,1989-10-05,1001001,0.000000,3.225777,0.872484
4,1989-10-06,1001001,0.000000,2.435284,0.872484
...,...,...,...,...,...
3627991,2008-12-27,12930001,0.958708,2.528415,3.410700
3627992,2008-12-28,12930001,1.995861,2.863661,3.816636
3627993,2008-12-29,12930001,0.975161,5.851183,4.172353
3627994,2008-12-30,12930001,1.066232,2.933977,4.226757


In [23]:
input_p.rename(columns = {'date':'Year_Mnth_Day', 'basin_id':'basin_id', 'precip_cr2met':'prcp(mm/day)', 'tmean_cr2met':'tmean(C)', 'streamflow_mm':'QObs(mm/d)'}, inplace = True)

In [24]:
input_p.sort_values(["Year_Mnth_Day", BASIN_COL], inplace=True, ignore_index=True)
input_p = input_p[~(input_p['Year_Mnth_Day'] < '1989-10-02')]
input_p = input_p[~(input_p['Year_Mnth_Day'] > '2008-12-31')]
input_p['Year_Mnth_Day'] = pd.to_datetime(input_p['Year_Mnth_Day'], format='%Y-%m-%d')

In [25]:
input_p['prcp(mm/day)'] = input_p['prcp(mm/day)'].fillna(input_p['prcp(mm/day)'].mean())
input_p['tmean(C)'] = input_p['tmean(C)'].fillna(input_p['tmean(C)'].mean())
input_p['QObs(mm/d)'] = input_p['QObs(mm/d)'].fillna(input_p['QObs(mm/d)'].mean())

In [26]:
assert len(input_p) == len (forcing_p) and len(input_p) == len (discharge_p)

In [27]:
BasicInputTimeSeries = input_p.to_numpy()

In [28]:
print(BasicInputTimeSeries.shape)
print(BasicInputTimeSeries[0])

(3627996, 5)
[Timestamp('1989-10-02 00:00:00') 1001001 0.0 3.7676558 0.86726993]


## Static Data

In [29]:
INVALID_ATTR= []

if CAMELS_COMBINED:
  INVALID_ATTR = [
      'gauge_name', 'p_mean_tmpa', 'gauge_record_start', 'gauge_record_end', 'gauge_n_obs', 'elev_gauge', 'record_period_start', 'record_period_end', 'n_obs', 'elev_med', 'elev_max', 'elev_min',
      'nested_inner', 'nested_outer', 'lc_barren', 'lc_glacier', 'p_mean_chirps', 'p_mean_mswep', 'location_type', 'geol_class_1st', 'geol_class_1st_frac',
      'geol_class_2nd', 'aridity_chirps', 'aridity_mswep', 'aridity_tmpa', 'p_seasonality_chirps', 'p_seasonality_mswep', 'p_seasonality_tmpa', 'frac_snow_chirps',
      'frac_snow_mswep', 'frac_snow_tmpa', 'high_prec_freq_chirps', 'high_prec_freq_mswep', 'high_prec_freq_tmpa', 'geol_class_2nd_frac', 'carb_rocks_frac', 'crop_frac',
      'grass_frac', 'high_prec_dur_chirps', 'high_prec_dur_mswep', 'high_prec_dur_tmpa', 'high_prec_timing_chirps', 'high_prec_timing_mswep', 'high_prec_timing_tmpa',
      'low_prec_freq_chirps', 'low_prec_freq_mswep', 'low_prec_freq_tmpa', 'low_prec_dur_chirps', 'low_prec_dur_mswep', 'low_prec_dur_tmpa', 'low_prec_timing_chirps',
      'low_prec_timing_mswep', 'low_prec_timing_tmpa', 'runoff_ratio_chirps', 'runoff_ratio_mswep', 'runoff_ratio_tmpa', 'stream_elas_chirps', 'stream_elas_mswep',
      'stream_elas_tmpa', 'big_dam', 'shrub_frac', 'wet_frac', 'imp_frac', 'barren_frac', 'snow_frac', 'fp_nf_index', 'land_cover_missing', 'glaciers_area', 'glaciers_frac',
      'p_mean_spread', 'swe_ratio', 'sur_rights_n', 'sur_rights_flow', 'interv_degree', 'gw_rights_n', 'gw_rights_flow', 'large_dam', 'dom_land_cover_frac', 'forest_frac',
      'slope_mean', 'nf_frac', 'fp_frac'
  ]

In [30]:
filtered_attr = attr_p.copy()
filtered_attr.drop(INVALID_ATTR, axis=1, inplace=True, errors='ignore')
filtered_attr.dtypes

gauge_id                     int64
gauge_lat                  float64
gauge_lon                  float64
area                       float64
elev_mean                  float64
dom_land_cover              object
p_mean_cr2met              float64
pet_mean                   float64
aridity_cr2met             float64
p_seasonality_cr2met       float64
frac_snow_cr2met           float64
high_prec_freq_cr2met      float64
high_prec_dur_cr2met       float64
high_prec_timing_cr2met     object
low_prec_freq_cr2met       float64
low_prec_dur_cr2met        float64
low_prec_timing_cr2met      object
q_mean                     float64
runoff_ratio_cr2met        float64
stream_elas_cr2met         float64
slope_fdc                  float64
baseflow_index             float64
hfd_mean                   float64
Q95                        float64
Q5                         float64
high_q_freq                float64
high_q_dur                 float64
low_q_freq                 float64
low_q_dur           

In [31]:
# Re-order Columns
filtered_attr = filtered_attr[['gauge_id', 'p_mean_cr2met', 'pet_mean', 'p_seasonality_cr2met', 'frac_snow_cr2met', 'aridity_cr2met',
                               'high_prec_freq_cr2met', 'high_prec_dur_cr2met', 'high_prec_timing_cr2met', 'low_prec_freq_cr2met',
                               'low_prec_dur_cr2met', 'low_prec_timing_cr2met', 'q_mean', 'runoff_ratio_cr2met', 'slope_fdc',
                               'baseflow_index', 'stream_elas_cr2met', 'Q5', 'Q95', 'high_q_freq', 'high_q_dur', 'low_q_freq',
                               'low_q_dur', 'zero_q_freq', 'hfd_mean', 'gauge_lat', 'gauge_lon', 'elev_mean', 'area', 'dom_land_cover']]

In [32]:
assert len(filtered_attr) == Nloc

In [33]:
# Categorical variable encoding
cols = filtered_attr.columns
categorical_cols = filtered_attr.select_dtypes(exclude = ['number']).columns

for cat_col in categorical_cols:
  num_cat = filtered_attr[cat_col].value_counts().count()
  filtered_attr[cat_col] = (filtered_attr[cat_col].astype('category').cat.codes)/num_cat

  print('Processed ', cat_col)

Processed  high_prec_timing_cr2met
Processed  low_prec_timing_cr2met
Processed  dom_land_cover


In [34]:
filtered_attr.isna().sum()

gauge_id                     0
p_mean_cr2met                0
pet_mean                     0
p_seasonality_cr2met         0
frac_snow_cr2met             0
aridity_cr2met               0
high_prec_freq_cr2met        0
high_prec_dur_cr2met         0
high_prec_timing_cr2met      0
low_prec_freq_cr2met         0
low_prec_dur_cr2met          0
low_prec_timing_cr2met       0
q_mean                     278
runoff_ratio_cr2met        278
slope_fdc                  278
baseflow_index             278
stream_elas_cr2met         278
Q5                         278
Q95                        278
high_q_freq                278
high_q_dur                 278
low_q_freq                 278
low_q_dur                  278
zero_q_freq                278
hfd_mean                   278
gauge_lat                    0
gauge_lon                    0
elev_mean                    0
area                         0
dom_land_cover               0
dtype: int64

In [35]:
# Remove NaN values
nan_cols = [col for col in filtered_attr.columns if filtered_attr.isna().sum()[col] > 0]
print(nan_cols)

for nan_col in nan_cols:
  filtered_attr[nan_col] = filtered_attr[nan_col].fillna(filtered_attr[nan_col].mean())

['q_mean', 'runoff_ratio_cr2met', 'slope_fdc', 'baseflow_index', 'stream_elas_cr2met', 'Q5', 'Q95', 'high_q_freq', 'high_q_dur', 'low_q_freq', 'low_q_dur', 'zero_q_freq', 'hfd_mean']


In [36]:
BasicInputStaticProps = filtered_attr.to_numpy()
NpropperTimeStatic = len(filtered_attr.columns) - 1
print('NpropperTimeStatic', NpropperTimeStatic)
print(filtered_attr.columns)

NpropperTimeStatic 29
Index(['gauge_id', 'p_mean_cr2met', 'pet_mean', 'p_seasonality_cr2met',
       'frac_snow_cr2met', 'aridity_cr2met', 'high_prec_freq_cr2met',
       'high_prec_dur_cr2met', 'high_prec_timing_cr2met',
       'low_prec_freq_cr2met', 'low_prec_dur_cr2met', 'low_prec_timing_cr2met',
       'q_mean', 'runoff_ratio_cr2met', 'slope_fdc', 'baseflow_index',
       'stream_elas_cr2met', 'Q5', 'Q95', 'high_q_freq', 'high_q_dur',
       'low_q_freq', 'low_q_dur', 'zero_q_freq', 'hfd_mean', 'gauge_lat',
       'gauge_lon', 'elev_mean', 'area', 'dom_land_cover'],
      dtype='object')


# Save Datasets

In [37]:
if CAMELS_COMBINED:
  input_p.to_csv(root_path + "/BasicInputTimeSeries_cl_combined.csv")
  filtered_attr.to_csv(root_path + "/BasicInputStaticProps_cl_combined.csv")
  np.save(root_path + "/BasicInputTimeSeries_cl_combined", BasicInputTimeSeries)
  np.save(root_path + "/BasicInputStaticProps_cl_combined", BasicInputStaticProps)

else:
  input_p.to_csv(root_path + "/BasicInputTimeSeries_cl.csv")
  filtered_attr.to_csv(root_path + "/BasicInputStaticProps_cl.csv")
  np.save(root_path + "/BasicInputTimeSeries_cl", BasicInputTimeSeries)
  np.save(root_path + "/BasicInputStaticProps_cl", BasicInputStaticProps)

In [38]:
from datetime import datetime
EndDate = datetime.strptime(EndDate, '%Y-%m-%d')
EndDate = np.datetime64(EndDate)
InitialDate = datetime.strptime(InitialDate, '%Y-%m-%d')
InitialDate = np.datetime64(InitialDate)

print(type(EndDate))
int(((EndDate-InitialDate + np.timedelta64(1, 'D'))/np.timedelta64(1, 'D')))

<class 'numpy.datetime64'>


7031

In [39]:
meta_data = {
    'Nloc': Nloc,
    'locs': locs.tolist(),
    'loc_names': attr_p['gauge_name'].tolist(),
    'BasicInputTimeSeries':{
      'fields': input_p.columns.values.tolist(),
      'index_fields': [BASIN_COL, DATE_COL],
      'initial_date': str(InitialDate),
      'end_date': str(EndDate),
      'time_delta': str(BasicInputTimeSeries[1, 1] - BasicInputTimeSeries[0, 1]),
      'time_steps': int(((EndDate-InitialDate + np.timedelta64(1, 'D'))/np.timedelta64(1, 'D'))),
    },
    'BasicInputStaticProps': {
        'fields': filtered_attr.columns.values.tolist(),
        'index_fileds': ['gauge_id'],
    },
    'NpropperTimeStatic': len(filtered_attr.columns) - 1
}

In [40]:
if CAMELS_COMBINED:
  with open(root_path + "/metadata_cl_combined.json", 'w') as outfile:
    json.dump(meta_data, outfile, indent='\t')

else:
  with open(root_path + "/metadata_cl.json", 'w') as outfile:
    json.dump(meta_data, outfile, indent='\t')

In [41]:
BasicInputTimeSeries

array([[Timestamp('1989-10-02 00:00:00'), 1001001, 0.0, 3.7676558,
        0.86726993],
       [Timestamp('1989-10-02 00:00:00'), 1001002, 0.0, 3.7481775,
        0.14062566],
       [Timestamp('1989-10-02 00:00:00'), 1001003, 0.0, 4.146216,
        0.018687354],
       ...,
       [Timestamp('2008-12-31 00:00:00'), 12876004, 0.55648845,
        7.5504074, 2.4411029994031685],
       [Timestamp('2008-12-31 00:00:00'), 12878001, 0.64957577,
        7.7898774, 0.5822323],
       [Timestamp('2008-12-31 00:00:00'), 12930001, 1.0372078, 5.933977,
        6.5284557]], dtype=object)

In [42]:
type(BasicInputTimeSeries)

numpy.ndarray

In [43]:
str(InitialDate)

'1989-10-02T00:00:00.000000'

In [44]:
str(BasicInputTimeSeries[1, 1] - BasicInputTimeSeries[0, 1])

'1'

In [45]:
meta_data

{'Nloc': 516,
 'locs': [1001001,
  1001002,
  1001003,
  1020002,
  1020003,
  1021001,
  1021002,
  1041002,
  1044001,
  1050002,
  1050004,
  1201001,
  1201003,
  1201005,
  1210001,
  1211001,
  1300009,
  1310002,
  1410004,
  1502002,
  1502008,
  1610002,
  1610004,
  1730001,
  1730002,
  1730003,
  1730007,
  1730012,
  2101001,
  2103001,
  2103002,
  2103003,
  2103014,
  2104002,
  2104003,
  2104013,
  2105001,
  2105002,
  2105005,
  2105007,
  2110001,
  2110002,
  2110004,
  2110031,
  2112005,
  2112006,
  2112007,
  2113001,
  2120001,
  2510001,
  3022001,
  3041001,
  3041002,
  3041003,
  3041004,
  3041005,
  3050001,
  3404001,
  3414001,
  3421001,
  3430001,
  3430002,
  3430003,
  3431001,
  3434001,
  3434003,
  3450001,
  3453001,
  3802001,
  3804002,
  3806001,
  3814001,
  3814003,
  3815001,
  3815002,
  3820001,
  3820002,
  3820003,
  3825001,
  3826001,
  4301002,
  4302001,
  4306001,
  4308001,
  4311001,
  4313001,
  4314001,
  4314002,
  4320001,